In [1]:
import pandas as pd
ad = pd.read_csv("Advertising.csv", usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [36]:
X = df.drop("sales", axis=1) # Bağımsız değişkenler seçildi
y = df["sales"] # Bağımlı değişkenler 

# Ayarlandı test ve train veri setleri
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [37]:
X_train.shape

(160, 3)

In [38]:
y_train.shape

(160,)

In [39]:
X_test.shape

(40, 3)

In [40]:
y_test.shape

(40,)

In [41]:
training = df.copy()  # lazım olur diye eski veri seti kopyalandı.
training.shape

(200, 4)

# Statsmodels

In [42]:
import statsmodels.api as sm
lm = sm.OLS(y_train, X_train)

In [43]:
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2935.
Date:                Tue, 16 Feb 2021   Prob (F-statistic):                   1.28e-137
Time:                        21:45:19   Log-Likelihood:                         -336.65
No. Observations:                 160   AIC:                                      679.3
Df Residuals:                     157   BIC:                                      688.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# scikit-learn model

In [47]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X_train, y_train)

In [48]:
model.intercept_

2.979067338122629

In [49]:
model.coef_

array([0.04472952, 0.18919505, 0.00276111])

# Tahmin
Model Denklemi:    
  
Sales = 2.97 + TV*0.04 + radio*0.18 + newspaper*0.002  
  
Örneğin 30 birim TV harcaması, 10 birim radio harcaması, 40 birim de gazete harcaması olduğunda satışların tahmini değeri ne olur?

In [50]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [51]:
yeni_veri = [[30], [10], [40]]
yeni_veri = pd.DataFrame(yeni_veri).T  # bu gerekli
yeni_veri

,0,1,2
0,30,10,40


In [52]:
model.predict(yeni_veri)   # Bu reklam harcamalarıyla tahmini satışımız bu olacak

array([6.32334798])

In [53]:
# Eğitim hatası,  model.predict(X_train) = tahmini y değerleri ile gerçek y değerleri karşılaştırılıyor.
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
rmse

1.644727765644337

In [54]:
# Test hatası, model eğitilmişti ama daha önce test değerlerini hiç görmemişti. şimdi bunla bakıyoruz
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

1.7815996615334502

# Model Tuning / Model Doğrulama

In [68]:
X = df.drop("sales", axis = 1)
y = df["sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state = 144)
lm = LinearRegression()
model = lm.fit(X_train, y_train)

In [69]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

1.6748559274650712

In [70]:
model.score(X_train, y_train)  # r^2 değeri

0.8971614078663419

**cros_Validation ile veriden random 10 tane veriseti seçilip bunlar üzerinde işlem yapılır ve bunların ortalaması alınarak valide edilmiş bir score elde edilir.
Daha doğru hata değeri elde edilmiş olur.**

In [71]:
cross_val_score(model, X, y, cv = 10, scoring = 'r2') # 10 katlı olsun, r^2 değerine bakmak istiyorum

array([0.87302696, 0.8581613 , 0.92968723, 0.89013272, 0.93146498,
       0.93138735, 0.7597901 , 0.91217097, 0.83891753, 0.92882311])

In [72]:
# 10 farklı random verialt seti için sonuç üretti şimdi bunların mean'ini alacağım. Modelin valide edilmiş hata değeri tüm veri seti çin
cross_val_score(model, X, y, cv = 10, scoring = 'r2').mean()

0.8853562237979616

In [73]:
cross_val_score(model, X_train, y_train, cv = 10, scoring = 'r2') # tran için yapalım

array([0.86580772, 0.92273806, 0.90690214, 0.9124702 , 0.69765146,
       0.92654282, 0.92526348, 0.94823778, 0.92941476, 0.69875488])

In [74]:
cross_val_score(model, X_train, y_train, cv = 10, scoring = 'r2').mean() # tran için yapalım

0.8733783298422942

In [77]:
#rmse değeri
np.sqrt(-cross_val_score(model, 
                X_train, 
                y_train, 
                cv = 10, 
                scoring = 'neg_mean_squared_error')).mean()

1.6649345607872927

In [79]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

1.6640263686701033

In [80]:
#rmse değeri
np.sqrt(-cross_val_score(model, 
                X_test, 
                y_test, 
                cv = 10, 
                scoring = 'neg_mean_squared_error')).mean()

1.7399924960346644